# レッスン 01  LLMの研究概要

このコースは、スタンフォード大学の公開講座であるCS336とCS224N、そしてAndrej KarpathyのYouTubeブログのコンテンツをまとめたものです。
お時間があれば、YouTubeでオリジナル動画、**特にCS336**の動画を視聴することを強くお勧めします。

<!-- ![CS336](https://github.com/yanwunhao/gonken-lesson-build-llm-from-scratch/blob/main/figs/cs336.png?raw=true) -->

## このコースを作った理由は？

8年前、研究者は自分でモデルを実装し訓練していた。

6年前、研究者はモデル（例：BERT）をダウンロードしてファインチューニングしていた。

現在、研究者はプロプライエタリなモデル（例：GPT-4/Claude/Gemini）にプロンプトを投げるだけ。

抽象度を上げることで生産性は向上するが、
* これらの抽象化には漏れがある（プログラミング言語やOSとは対照的に）。
* スタックを解体する必要がある基礎研究がまだ残っている。

この技術の**完全な理解**は研究に必要である。

しかし、現在、ほとんどの研究機関は非常に現実的な問題に直面しています。それが**言語モデルの産業化**です。

* GPT-4は推定1.8兆パラメータを持つとされる。
* GPT-4の訓練には推定1億ドルかかったとされる。
* xAIはGrokを訓練するために20万台のH100でクラスターを構築。
* Stargate（OpenAI、NVIDIA、Oracle）は4年間で5000億ドルを投資

![stargate](https://github.com/yanwunhao/gonken-lesson-build-llm-from-scratch/blob/main/figs/sb_stargate.png?raw=true)





また、フロンティアモデルがどのように構築されているかについて、公開されている詳細は存在しない。

GPT-4技術レポートより [Achiam, Josh, et al. "Gpt-4 technical report." arXiv preprint arXiv:2303.08774 (2023).](https://arxiv.org/abs/2303.08774)

**“競争環境と、GPT-4のような大規模モデルの安全性への影響の両方を考慮し、本レポートにはアーキテクチャ（モデルサイズを含む）、ハードウェア、訓練計算量、データセット構築、訓練方法、またはその他類似の詳細については一切含まれていない”**

![gpt_no_detial](https://github.com/yanwunhao/gonken-lesson-build-llm-from-scratch/blob/main/figs/gpt4-no-details.png?raw=true)

## Section 1 More is different

フロンティアモデルは学校の研究者たちの手の届かないところにある。

しかし、小規模言語モデルを構築することは、大規模言語モデルを代表するものではないかもしれない。

Example 1: アテンション対MLPで費やされるFLOPsの割合はスケールとともに変化する。

![roller](https://github.com/yanwunhao/gonken-lesson-build-llm-from-scratch/blob/main/figs/roller-flops.png?raw=true)

Example 2: スケールに伴う振る舞いの創発

[Wei, Jason, et al. "Emergent abilities of large language models." arXiv preprint arXiv:2206.07682 (2022).](https://arxiv.org/abs/2206.07682)

![wei](https://github.com/yanwunhao/gonken-lesson-build-llm-from-scratch/blob/main/figs/wei-emergence-plot.png?raw=true)


## Section 2 学校で学べることのうち、産業界のモデルに転用できるものは何か？

知識には3つのタイプがある：

* メカニクス（仕組み）：物事がどう機能するか（Transformerとは何か、モデル並列化がGPUをどう活用するか）
* マインドセット（思考法）：ハードウェアから最大限を引き出すこと、スケールを真剣に捉えること（スケーリング則）
* 直感：どのデータとモデリングの決定が良い精度をもたらすか

メカニクスとマインドセットは転用可能

直感は部分的にしか学校で教えられない（スケール間で必ずしも転用できるとは限らない）。

直感とは？

一部の設計判断は単に（まだ）正当化できず、実験から得られたものに過ぎない。

Example: SwiGLU （From Google）

[Shazeer, Noam. "Glu variants improve transformer." arXiv preprint arXiv:2002.05202 (2020).](https://arxiv.org/abs/2002.05202)

**”私たちは、これらのアーキテクチャがなぜ機能するように見えるのかについて、何の説明も提供しない。その成功は、他のすべてと同様に、神の慈悲によるものだと考えている。”**

![SwiGLU](https://github.com/yanwunhao/gonken-lesson-build-llm-from-scratch/blob/main/figs/divine-benevolence.png?raw=true)

## Section 3 LLMにとって効率は一番大事なこと

精度 = 効率 × 計算リソース

実際、効率性は”大規模”言語モデルになるほど遥かに重要である（無駄にする余裕はない）。

Hernandez, Danny, and Tom B. Brown. "Measuring the algorithmic efficiency of neural networks." arXiv preprint arXiv:2005.04305 (2020). (From OpenAI)

2012年から2019年の間にImageNetで44倍のアルゴリズム効率を示した。

フレーミング：特定の計算量とデータ予算で構築できる最良のモデルは何か？

言い換えれば、効率を最大化する！

[Liu, Aixin, et al. "Deepseek-v3 technical report." arXiv preprint arXiv:2412.19437 (2024).](https://arxiv.org/abs/2412.19437)

DeepSeek-V3の訓練エポック

**事前訓練段階（Pre-training）**

訓練データ量は14.8兆トークン
具体的なエポック数は明記されていないが、データ量から見て1エポック未満の可能性が高い。

**教師ありFine-tuning段階（SFT）**

明確に2エポック訓練
コサイン減衰学習率を使用、5×10⁻⁶から1×10⁻⁶に減少。

訓練コスト
DeepSeek-V3の完全な訓練には278.8万H800 GPU時間のみ必要で、1時間あたり2ドルで計算すると、総コストは約558万ドル。

## Question：1024台のH100で70Bパラメータモデルを15Tトークンで訓練するのにどれくらい時間がかかるか？

In [4]:
total_flops = 6 * 70e9 * 15e12 # @inspect total_flops
h100_flop_per_sec = 1979e12 / 2
mfu = 0.5
flops_per_day = h100_flop_per_sec * mfu * 1024 * 60 * 60 * 24 # @inspect flops_per_day
days = total_flops / flops_per_day # @inspect days

print(days)

143.92659381842682


## Section 4 Scaling laws

問題：FLOPs予算が与えられたとき、より大きなモデル(D)を使用するか、より多くのトークン(N)で訓練するか？

計算最適Scaling laws: 

[Kaplan, Jared, et al. "Scaling laws for neural language models." arXiv preprint arXiv:2001.08361 (2020).](https://arxiv.org/abs/2001.08361)

[Hoffmann, Jordan, et al. "Training compute-optimal large language models." arXiv preprint arXiv:2203.15556 (2022).](https://arxiv.org/abs/2203.15556)

![scaling_laws](https://github.com/yanwunhao/gonken-lesson-build-llm-from-scratch/blob/main/figs/chinchilla-isoflop.png?raw=true)

**D=20N**（例：14億パラメータモデルは280億トークンで訓練すべき）

# Section 5 **nanoGPT**: 簡潔で完全なGPT実装

Andrej Karpathyが開発したnanoGPTは、教育用に設計されたGPT-2の再現プロジェクトです：
* コード量は500行未満だが、完全な訓練パイプラインを含む
* 単一GPUで、連続したテキストを生成できるモデルを訓練可能
* 「ゼロから」言語モデルを構築する核心原理を体現

GitHub: https://github.com/karpathy/nanoGPT

**次回以降のレッスン（３－４回）では、nanoGPTをステップバイステップで一緒に実装していきます。Tokenizerの原理やByte Pair Encoding（BPE）、Transformerの各層の実装、訓練ループの構築など、コードを書きながら深く理解していきましょう。**